# <center> Рекомендательные системы </center>

## <center> Домашнее задание 1 </center>

## Автор материала: доцент ФКН НИУ ВШЭ Дмитрий Игнатов
Можно использовать материал в любых целях, кроме коммерческих. Примеры составлены по мотивам главы 2 книги Т.Сегаран "Программируем коллективный разум" http://www.symbol.ru/alphabet/613828.html

### Срок выполнения: 19 ноября 2020 23:00




In [1]:
import pickle
from math import sqrt
from tqdm import tqdm

## Словарь с предпочтениями {кинокритик: {фильм : оценка}}

In [2]:
with open('data/critics_reviews', 'rb') as f:
    critics = pickle.load(f)

display(critics)

{'Lisa Rose': {'Lady in the Water': 2.5,
  'Snakes on a Plane': 3.5,
  'Just My Luck': 3.0,
  'Superman Returns': 3.5,
  'You, Me and Dupree': 2.5,
  'The Night Listener': 3.0},
 'Gene Seymour': {'Lady in the Water': 3.0,
  'Snakes on a Plane': 3.5,
  'Just My Luck': 1.5,
  'Superman Returns': 5.0,
  'The Night Listener': 3.0,
  'You, Me and Dupree': 3.5},
 'Michael Phillips': {'Lady in the Water': 2.5,
  'Snakes on a Plane': 3.0,
  'Superman Returns': 3.5,
  'The Night Listener': 4.0},
 'Claudia Puig': {'Snakes on a Plane': 3.5,
  'Just My Luck': 3.0,
  'The Night Listener': 4.5,
  'Superman Returns': 4.0,
  'You, Me and Dupree': 2.5},
 'Mick LaSalle': {'Lady in the Water': 3.0,
  'Snakes on a Plane': 4.0,
  'Just My Luck': 2.0,
  'Superman Returns': 3.0,
  'The Night Listener': 3.0,
  'You, Me and Dupree': 2.0},
 'Jack Matthews': {'Lady in the Water': 3.0,
  'Snakes on a Plane': 4.0,
  'The Night Listener': 3.0,
  'Superman Returns': 5.0,
  'You, Me and Dupree': 3.5},
 'Toby': {'Snak

In [3]:
# Вычисление расстояния Евклида
sqrt(pow(5 - 4, 2) + pow(4 - 1, 2))

3.1622776601683795

In [4]:
# Вычисление сходства
1/(1 + sqrt(pow(5 - 4, 2) + pow(4 - 1, 2)))

0.2402530733520421

In [5]:
def sim_distance(prefs, person1, person2):
    '''
    Возвращает сходство критиков на основе расстояния.
    '''
    # Получить список предметов, оцененных обоими
    similar = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            similar[item] = 1
    # Если нет ни одной общей оценки, вернуть 0
    if len(similar) == 0:
        return 0
    # Сложить квадраты разностей
    sum_of_squares = sum([(prefs[person1][item] - prefs[person2][item])**2
                         for item in prefs[person1] if item in prefs[person2]])
    return 1/(1 + sum_of_squares)

In [6]:
sim_distance(critics, 'Lisa Rose','Gene Seymour')

0.14814814814814814

In [7]:
def sim_pearson(prefs, person1, person2):
    '''
    Возвращает коэффициент корреляции Пирсона между двумя критиками.
    '''
    # Получить список предметов, оцененных обоими
    similar = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            similar[item] = 1
    # Если нет ни одной общей оценки, вернуть 0
    if len(similar) == 0:
        return 0
    # Количество соместно оцененных фильм
    n = len(similar)
    # Вычислить сумму всех предпочтений
    sum1 = sum([prefs[person1][i] for i in similar])
    sum2 = sum([prefs[person2][i] for i in similar])
    # Вычислить сумму квадратов
    sum1_sqrt = sum([prefs[person1][i]**2 for i in similar])
    sum2_sqrt = sum([prefs[person2][i]**2 for i in similar])
    # Вычислить сумму произведений
    sum_product = sum([prefs[person1][i]*prefs[person2][i] for i in similar])
    # Вычислить коэффициент Пирсона
    num = sum_product - (sum1*sum2/n)
    den = sqrt((sum1_sqrt - sum1**2/n)*(sum2_sqrt-sum2**2/n))
    if den == 0:
        return 0
    r_coeff = num/den
    return r_coeff

In [8]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

## Ранжирование критиков

In [9]:
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    '''
    Возвращает список наилучших соответствий для человека из словаря prefs.
    Длина списка и функция сходства – необязательные параметры.
    '''
    scores = [(similarity(prefs, person, other), other) for other in prefs if other != person]
    # Предпочтения по убыванию оценок
    scores.sort(reverse=True)
    return scores[:n]

In [10]:
topMatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

## Рекомендация фильмов (User-based подход)

In [11]:
def getRecommendations(prefs, person, similarity=sim_pearson):
    '''
    Получить рекомендации для заданного человека, пользуясь взвешенным средним
    оценок от всех остальных пользователей.
    '''
    totals = {}
    sim_sums = {}
    for other in prefs:
        # Сравнивать меня с собой же не нужно
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        # Будем игнорировать нулевую и отрицательную схожесть
        if sim <= 0:
            continue
        for item in prefs[other]:
            # Оценивать только фильмы, которые человек еще не смотрел
            if item not in prefs[person] or prefs[person][item] == 0:
                # Коэффициент схожести * Оценка
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item]*sim
                # Сумма коэффициентов схожести
                sim_sums.setdefault(item, 0)
                sim_sums[item] += sim
    # Создать нормированный список
    rankings = [(total/sim_sums[item], item) for item, total in totals.items()]
    # Вернуть отсортированный список
    rankings.sort(reverse=True)
    return rankings

In [12]:
getRecommendations(critics, 'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [13]:
getRecommendations(critics, 'Toby', sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

## Сходство предметов

Хотим заменить

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}

на

{'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},
'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}?


In [14]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # Обменять местами человека и предмет
            result[item][person] = prefs[person][item]
    return result

In [15]:
movies = transformPrefs(critics)
movies

{'Lady in the Water': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.0,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0},
 'Snakes on a Plane': {'Lisa Rose': 3.5,
  'Gene Seymour': 3.5,
  'Michael Phillips': 3.0,
  'Claudia Puig': 3.5,
  'Mick LaSalle': 4.0,
  'Jack Matthews': 4.0,
  'Toby': 4.5},
 'Just My Luck': {'Lisa Rose': 3.0,
  'Gene Seymour': 1.5,
  'Claudia Puig': 3.0,
  'Mick LaSalle': 2.0},
 'Superman Returns': {'Lisa Rose': 3.5,
  'Gene Seymour': 5.0,
  'Michael Phillips': 3.5,
  'Claudia Puig': 4.0,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 5.0,
  'Toby': 4.0},
 'You, Me and Dupree': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.5,
  'Claudia Puig': 2.5,
  'Mick LaSalle': 2.0,
  'Jack Matthews': 3.5,
  'Toby': 1.0},
 'The Night Listener': {'Lisa Rose': 3.0,
  'Gene Seymour': 3.0,
  'Michael Phillips': 4.0,
  'Claudia Puig': 4.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0}}

In [16]:
topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [17]:
getRecommendations(movies,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

In [18]:
def calculateSimilarItems(prefs, n=10):
    '''
    Создать словарь, содержащий для каждого объекта наиболее похожие на него n объеков.
    '''
    result = {}
    # Обратить матрицу предпочтений, чтобы строки соответствовали фильмам
    itemPrefs = transformPrefs(prefs)
    counter = 0
    # Добавим progress bar для больших датасетов
    for item in tqdm(itemPrefs):
        # Найти объекты, максимально похожие на данный
        scores = topMatches(itemPrefs, item, n=n, similarity=sim_distance)
        result[item] = scores
    return result

In [19]:
len(movies.keys())

6

У нас всего 6 фильмов, посчитаем скоры похожести для всех пар:

In [20]:
itemsim = calculateSimilarItems(critics, n=5)
itemsim

100%|██████████| 6/6 [00:00<00:00, 14665.40it/s]


{'Lady in the Water': [(0.4, 'You, Me and Dupree'),
  (0.2857142857142857, 'The Night Listener'),
  (0.2222222222222222, 'Snakes on a Plane'),
  (0.2222222222222222, 'Just My Luck'),
  (0.09090909090909091, 'Superman Returns')],
 'Snakes on a Plane': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'The Night Listener'),
  (0.16666666666666666, 'Superman Returns'),
  (0.10526315789473684, 'Just My Luck'),
  (0.05128205128205128, 'You, Me and Dupree')],
 'Just My Luck': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'You, Me and Dupree'),
  (0.15384615384615385, 'The Night Listener'),
  (0.10526315789473684, 'Snakes on a Plane'),
  (0.06451612903225806, 'Superman Returns')],
 'Superman Returns': [(0.16666666666666666, 'Snakes on a Plane'),
  (0.10256410256410256, 'The Night Listener'),
  (0.09090909090909091, 'Lady in the Water'),
  (0.06451612903225806, 'Just My Luck'),
  (0.05333333333333334, 'You, Me and Dupree')],
 'You, Me and Dupree': [(0.4,

In [21]:
def getRecommendedItems(prefs, itemMatch, user):
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # Цикл по образцам, оцененным данным пользователем
    for item, rating in userRatings.items():
        # Цикл по образцам, похожим на данный
        for similarity, item in itemMatch[item]:
            # Пропускаем, если пользователь уже оценивал данный образец
            if item in userRatings: 
                continue
            # Взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item, 0)
            scores[item] += similarity * rating
            # Сумма всех коэффициентов подобия
            totalSim.setdefault(item, 0)
            totalSim[item] += similarity
            if totalSim[item] == 0: 
                totalSim[item] = 0.0000001 # чтобы избежать деления на ноль
    # Делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить среднее
    rankings = [(score/totalSim[item], item) for item, score in scores.items()]
    # Возвращает список rankings, отсортированный по убыванию
    rankings.sort(reverse=True)
    return rankings

In [22]:
getRecommendedItems(critics,itemsim,'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

## Рекомендация на данных MovieLens

Источник: http://grouplens.org/datasets/movielens/

In [23]:
def loadMovieLens(path='data/movielens'):
    # Получить названия фильмов
    movies = {}
    for line in open(path + '/u.item', errors='ignore'):
        idx, title = line.split('|')[:2]
        movies[idx] = title
    # Загрузить данные
    prefs = {}
    for line in open(path + '/u.data', errors='ignore'):
        user, movie_id, rating, ts = line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movie_id]] = float(rating)
    return prefs

In [24]:
prefs = loadMovieLens()
prefs['87'] # user_id

{'Naked Gun 33 1/3: The Final Insult (1994)': 4.0,
 'Con Air (1997)': 4.0,
 'Sabrina (1995)': 4.0,
 'Waterworld (1995)': 4.0,
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)': 3.0,
 'Clueless (1995)': 4.0,
 'Jurassic Park (1993)': 5.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Son in Law (1993)': 4.0,
 'Indiana Jones and the Last Crusade (1989)': 5.0,
 'Good, The Bad and The Ugly, The (1966)': 5.0,
 'Dead Poets Society (1989)': 5.0,
 'Dead Man Walking (1995)': 4.0,
 "Joe's Apartment (1996)": 2.0,
 'GoldenEye (1995)': 4.0,
 'M*A*S*H (1970)': 5.0,
 'Something to Talk About (1995)': 2.0,
 'Lightning Jack (1994)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Cowboy Way, The (1994)': 3.0,
 "Ulee's Gold (1997)": 3.0,
 'Addams Family Values (1993)': 2.0,
 '2001: A Space Odyssey (1968)': 5.0,
 'Platoon (1986)': 3.0,
 'Return of the Pink Panther, The (1974)': 4.0,
 'Four Weddings and a Funeral (1994)': 5.0,
 'Under Siege (1992)': 4.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Die Hard: Wit

In [25]:
itemsim = calculateSimilarItems(prefs, n=50)

100%|██████████| 1664/1664 [00:31<00:00, 52.37it/s] 


In [26]:
getRecommendedItems(prefs, itemsim, '87')[0:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'House of the Spirits, The (1993)'),
 (5.0, 'Fresh (1994)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Before the Rain (Pred dozhdot) (1994)'),
 (5.0, 'Assignment, The (1997)'),
 (5.0, '1-900 (1994)'),
 (4.888888888888889, "Ed's Next Move (1996)"),
 (4.833333333333333, 'Anna (1996)'),
 (4.8, 'Dark City (1998)'),
 (4.77777777

# Задание 1. Сравнение методов коллаборативной фильтрации по сходству пользователей и по сходству объектов


1. Требуется реализовать вычисление ошибки **MAE** и **RMSE** на тестовых данных [MovieLens](http://grouplens.org/datasets/movielens/).  
В качестве данных обучения можно использовать файлы с расширением base, а тестирование качества провести на файле test: пары файлов u1.base и u1.test, ..., u5.base и u5.test. Каждая пара -- это разбиение данных на обучающие (80%) и тестовые (20%) данные для всех пользователей $u$.
2. Для каждого метода (user-based и item-based) постройте графики зависимости MAE и RMSE от числа соседей (диапазон от 1 до 100 с разумным шагом).
3. Если качество предсказаний слишком низкое (MAE > 2.0), попробуйте формулы 2.6 и 2.7 из обзора http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf.
Можно использовать альтернативные формулы для исходной модели $r_{u,i} = k\cdot \sum\limits_{u^\prime \in U}\operatorname{sim}(u,u^\prime) \cdot r_{u^\prime, i} \mbox{ (случай user-based модели):}$
$$r_{u,i} = \frac{1}{N}\sum\limits_{u^\prime \in U}r_{u^\prime, i}$$
$$r_{u,i} = \bar{r_u} +  k\cdot \sum\limits_{u^\prime \in U} sim(u,u^\prime)(r_{u^\prime, i}-\bar{r_{u^\prime}} ) \mbox{, где } k =\frac{1}{\sum_{u^\prime \in U}|\operatorname{sim}(u,u^\prime)|}$$
4. Сравните подходы на основе полученных результатов по аналогии с пунктами 1 и 2. 
5. Как изменяется величина MAE (RMSE) от числа выдаваемых рекомендаций (top-n): $n \in \{1,3,5,10,15,20,30,40,50,100\}$? 
6. Как Вы считаете, какие фильмы чаще рекомендуются - популярные с высокими оценками или редкие (те, которые редко оцениваются) с высокими оценками? Почему это происходит?
7. Что делать, если соседей (то есть, похожих на целевого пользователя или конкретный товар) мало? Нужно (и можно) ли как-то учитывать достоверность таких рекомендаций? 
8. *Необязательное задание*. Насколько различны списки из top-n рекомендаций. Попробуйте улучшить результаты подбором $\beta$ для минимизации MAE (RMSE) в гибридных рекомендациях в зависимости от числа соседей:
$$\beta\cdot r^{user-based}_{ui} + (1-\beta)\cdot r^{item-based}_{ui}, \mbox{ где } 0 \leq \beta \leq 1.$$ 

